# Prompt Engineering

In [ ]:
import os
import openai
openai.api_key = '' # 显然我们并没有 API KEY

def get_completion(prompt, model='gpt-3.5-turbo', temperature=0):
    message = [{
        'role': 'user',
        'content': prompt
    }]
    response = openai.ChatCompletion.create(
        model=model,
        message=message,
        temperature=0   # degree of randomness
    )
    return response.choices[0].message['content']


## 1 两个基本原则

模型存在的局限（幻觉）：

- 产出一些听起来很合理，但实际上错误的答案
  
- 你可以尝试：让模型找到参考文献，再根据参考文献回答（然后验证 ref 的真实性）

### 1.1 编写明确且具体的指令

- 使用 **分隔符** 清楚地划分 **输入的不同部分**

    事实上，使用 **分隔符** 还能避免模型执行用户文本中携带的错误操作（类似于 SQL 注入）

- 要求模型进行 **结构化** 输出：HTML or JSON

- 要求模型检查 **是否满足指定条件**：若不满足，则立即停止生成

- Few-shot Prompting: 在模型执行任务前题懂 **成功** 执行任务的示例

In [4]:
# use sample: for Summary task
# 对于指定 prompt，我们只需要调用 get_completion(prompt) 即可获取回应文本

text = f"... a long text"
# 此处使用 三个反引号 ``` 对输入成分进行分割
prompt = f"""
Summarize the text delimited by triple backticks into a single sentence.
```{text}```
"""
response = get_completion(prompt)

APIRemovedInV1: 

You tried to access openai.ChatCompletion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742


In [ ]:
# use sample: for Generation task
# 此处指定了输出的 JSON 格式
prompt = f"""
Generate a list of three made-up book titles along with their authors and genres.
Provide them in JSON format with the following keys:
book_id, title, author, genre.
"""
response = get_completion(prompt)

In [ ]:
# use sample: for Step Extraction task

text = f"一段用于描述过程的长文本"
# 此处考虑了输入 text 中"并不包含步骤描述"的边界条件
prompt = f"""
You will provided with text delimited by triple quotes.
If it contains a sequence of instructions, \
re-write those instructions in the following forms:

Step 1 - ...
Step 2 - ...
...
Step N - ...

If teh text does not contain a sequence of instructions, \
then simply write \"No steps provided.\"

\"\"\"{text}\"\"\"
"""

In [ ]:
# use sample: Few-shot prompting
# 给出了 grandparent 的排比输出样例
prompt = f"""
Your task is to answer in a consistent style.

<child>: Teach me about patience.

<grandparent>: The river that carves the deepest valley \
flows from a modest spring; the grandest symphony originates \
from a single note; the most intricate tapestry begins with a solitary thread.

<child>: Teach me about resilience.
"""

### 2.2 给模型足够的时间去思考

- 指定完成任务所需的步骤

- 让模型在下结论之前先给出自己的解法、再与输入的方案进行比较

In [ ]:
# use sample: Specify the Steps
text = f"一段长文本"
# prompt 中给出的 summary -> translate -> list name 的具体步骤s
prompt = f"""
Perform the following actions:
1 - Summarize the following text delimited by triple backticks with 1 sentence.
2 - Translate the summary into French.
3 - List each name in the French summary.
4 - Output a JSON object that continas the following keys: \
    french_summary, num_names.

Separate your answers with line breaks.

Text:
```{text}```
"""

# 你也可以具体要求模型以特殊格式进行输出
prompt = f"""
Your task is to perform the following actions:
1 - Summarize the following text delimited by <> with 1 sentence.
2 ~ 4 是一样的

Use the following format:
Text: <tetx to summarize>
Summary: <summary>
Translation: <summary translation>
Names: <list of names in Italian summary>
Output JSON: <JSON with summary and num_names>

Text to summmarize: <{text}>
"""

In [ ]:
# use sample: is Student's answer correct?
text = f"反正是学生的解法"
prompt = f"""
Determine if the student's solution is correct or not.

Question:
I'm building a solar power installation and I need help working out the financials.
- Land costs $100 / square foot
- I can buy solar panels for $250 / square root
- I negotiated a contract for maintenance that will cost me a flat $100k per year, \
and an additional $10 / square foot
What is the total cost for the first year of operations as \
a function of the number of square feet.

Student's Solution:
{text}
"""

# 直接输出结果 -> 模型会认为学生的解法正确（实际上是错的）
# => 我们应该让模型先生成自己的解法、再与学生的进行对比
prompt = f"""
Your task is to determine if the student's solution is correct or not.
To solve the problem do the following:
- First, work out your own solution to the problem.
- Then, compare your solution to the student's solution \
and evaluate if the student's solution is correct or not.
Don't decide if the student's solution is correct until you have doen the problem yourself.

Use the following format:
Question:
```
    question here
```
Student's solution:
```
    student's solution here
```
Actual solution:
```
    steps to work out the solution and your solution here
```
Is the student's solution the same as actual solution just calculated:
```
    yes or no
```
Student grade:
```
    correct or incorrect
```

Question:
```
{question}
```
Student's solution:
```
{solution}
```
Actual solution:
"""

## 2 迭代

In [ ]:
fact_chair_sheet = f"关于一把椅子的产品信息"

# 基于技术说明书撰写产品说明
# 因为 LLM 底层用了 tokenizer，所以直接限制 word / character 数量的效果不是很好s
prompt = f"""
Your task is to help a marketing team create a description for a retail website \
of a product based on a technical face sheet.

Write a product description based on the information \
provided in the technical specifications delimited by triple backticks.

# 需要在描述的末尾输出商品ID
At the end of the description, include every 7-character Product ID in the technical specification.

# 增加 toB 的限制描述
The description is intended for furniture relailers, \
so should be technical in nature and forcus on the materials the produt is constructed from.

# 限制字数
Use at most 50 words. / Use at most 3 sentences.

# 让 GPT 以 HTMl 格式组织回答
Format everything as HTML that can be used in a website.
Place the description in a <div> element.

Technical specifications: ```{fact_chair_sheet}```
"""

## 3 摘要

让我们用 LLM 总结文本吧！

In [ ]:
prod_review = f"只是一段长评论"

prompt = f"""
Your task is to generate a short summary of a product review \
# 你也可以把 summary 替换成信息提取任务
Your task is to extract relevant information from a product reivew \
# 限定适用范围
from an ecommerce site \
to give feedbackto the Shipping department.

Summarize the review below, delimited by triple backticks, \
# 限定长度
in at most 30 words, \
# 进一步强调 shippin' 信息
and focusing on any aspect that mention shipping and delivery of the product.

Review: ```{prod_review}``
"""

## 4 推理

我们可以把这些任务视为以 text 为输入的 infer 任务，例如：标签提取、情感分析 ...

In [5]:
review = f"反正是一个长评论"

# 积极/消极情感 分类
prompt = f"""
What is the sentiment of the following product review,
which is delimited with triple backticks?

# 只输出单个词汇
Give your answer as a single word, either 'positive' or 'negative'.

Review text: ```{review}```
"""

# 特定情感分类：用户是否愤怒
prompt = f"""
If the writer of the following review expressing anger? \
The review is delimited with triple backticks. \
Give your answer as either yes or no.
...
"""

In [ ]:
# 情感标签提取（<=5个）
prompt = f""" 
Identify a list of emotions that the writer of the following review \
is expressing. Include no more than 5 items in the list. \
Format your answer as a list of lower-case words separated by commas.
...
"""

# 具体信息提取
prompt = f"""
Identify the following items from the review text:
- Item purchased by reviewer
- Company that made the item

The review is delimited with three backticks. \
Format your response as a JSON object with \
'Item' and 'Brand' as the keys.
If the information isn't present, use 'unknown' as the value.

Make your response as shory as possible.

Review text:```{text}```
"""

In [ ]:
# 事实上你可以通过一条 prompt 同时完成上述的四个任务
prompt = f""" 
# 这里塞一坨任务
Identify the following items from the review text:
- Sentiment (positive or negative)
- Is the reviewer expressing anger? (ture or false)
- Item purchased by reviewer
- Company that made the item

# 这里规定输出格式
The reveiw is delimited with 3 backticks. \
Format your response as a JSON object with \
'Sentiment', 'Anger', 'Item' and 'Brand' as the keys. \
If the information isn't present, use 'unknown' as the value. \

Make your response as short as possible.

Format the Anger value as boolean.
"""

In [ ]:
# 类似的，我们可以进行文本的主题提取
prompt = f"""
Detemnine 5 topics that are being discussed in the following text, \
which is deliemited by 3 backticks.

Make each item 1 or 2 words long.

Format your response as a list of items separated by commas.
"""

# 你也可以让模型判断输入是否包含指定的主题（Zero-shot）
prompt = f""" 
Determine whether each item in the following list of topics \
is a topic in the text below, which is delimited with 3 backticks.

Give your answer as list with 0 or 1 for each topic. \
List of topics: {topics}

Text sample: ```{text}```
"""

NameError: name 'topics' is not defined

## 5 转换

将输入转变为不同格式，belike：翻译，润色，语法纠正，从 HTML 输入转换到 JSON ...

In [ ]:
# 翻译（至多种语言）
prompt = f"""" 
Translate the following [language A] text to [language B] and [languate C] ...: \
```{text}```
"""

# 你也可以指定语气
"Translate the following text to XXX in both the formal and informal form: "

# 识别源语言
prompt = f""" 
Tell me which language this is: \
```{text}```
"""

In [ ]:
# 好的，现在我们可以构建一个通用的 多源语言 -> 指定语言 翻译器
for msg in messages:
    # 1 识别源语言 
    prompt = f""" 
    Tell me which language this is: \
    ```{msg}```
    """
    lan = get_completion(prompt)

    prompt = f"""" 
    Translate the following {lan} text to [target_lan]: \
    ```{msg}```
    """
    res = get_completion(prompt)


In [13]:
# 语气转换 => 更商业的？
prompt = f""" 
Translate the following from slang to a business letter:
```{text}```
"""

In [ ]:
# 格式转换：你需要描述输入输出格式
data_json = {'employees': [
    {'name': 'Shyam', 'email': 'xxx.com'}
]}

# JSON -> HTMl
prompt = f""" 
Translate the following python dictionary from JSON to HTML table \
with column headers and title: {data_json}
"""

你可以通过 RedLines 包来可视化两段文字之间的差异！

In [ ]:
# 拼写和语法检查
prompt = f""" 
Proofread and correct the following text, and rewrite the corrected version. \
If you don't find any errors, just say 'No error found':
```{sentence}```
"""

# 润色至指定格式：符合 APA 样式，且面向高级用户
prompt = f""" 
Proofread and correct this review. Make it more compelling. \
Ensure it follows APA style and targets an advanced reader.

Output in markdown format.
"""

## 6 扩展

将短文本（一组说明或主题列表）转换为长文本（如电子邮件或某个主题的文章）

Temperature（奇妙超参数）：类似于随机性
> 感觉是对热运动的 neta？温度越高，随机性越强

In [ ]:
# use sample: 根据文本情感自动生成回复邮件
review = f'这事用户评论'
sentiment = 'negative' # 用前面介绍的方法提取的

# 总的来说是一种 role play
prompt = f""" 
You are a customer service AI assistant.

Your task is to send an email reply to a valued customer.
Given the customer email, delimited by ```, \
generate a reply to thank the customer for thier review.

If the sentiment is positive or neutral, thank them for their review.
If the sentiment is negative, apologize and suggest that they can reach out to customer service.

Make sure to user specific details from the review.
Write in a concise and professional tone.
Sign the email as `AI customer agent`.

Customer review: ```{review}```
Review sentiment: {sentiment}
"""

## 7 聊天机器人

In [ ]:
# 一个新的函数！
def get_completion_from_message(message, model='gpt-3.5-turbo', temperature=0):
    response = openai.ChatCompletion.create(
        model=model,
        message=message,
        temperature=temperature
    )
    return response.choices[0].message['content']

"""
实际上完整的 response 长这样：
{
    'content': '回应文本',  # 我们的函数只返回了这部分
    'role':    '角色'      # 一般是 assistant
}
"""

好的，那么 `msg` 和 `prompt` 有什么区别呢？

- 这是一条 `prompt`：

    ```python
    prompt = '这是一句话'
    ```

- 而这是一份 `message`:

    ```python
    message = {
        'role': 'user',
        'content': prompt
    }
    ```

没错，`msg` 比 `prompt` 多了一个 ‘role’ 标签，用以区分消息的不同主体。

In [ ]:
# 然后，我们就可以丢一串“多用户”的连续文本啦！
messages = [
    {
        'role': 'system',
        'content': 'You are an assistant ...'
    },{
        'role': 'user',
        'content': 'Tell me a joke.'
    },{
        'role': 'assistent',
        'content': 'Why did the chicken ...'
    },{
        'role': 'user',
        'content': 'What is the meaning?'
    }
]

# 把这一坨 msg 丢给 GPT，他会给你返回下一条 assistant 信息

其中：

- `User` 为用户输入的文本
- `Assitant` 为 GPT 输出的文本
- `System` 则是认为设置的、**用于规定 assistant 行为的文本**（setting），用户 **不能** 看见系统消息

理想的消息序列是：`<sys>`, `<usr>`, `<gpt>`, `<usr>`, `<gpt>`, ...
    

但是，每次给 LLM 丢 msg 序列都是一次 **独立** 的交互！

=> 这意味着：如果你希望模型记住过去的信息，你必须 **提供上下文**

=> 好吧，其实就是 **把前面的对话全都喂回去**（所以 context 会持续变长...）

In [ ]:
# 好的，这是一个自动把新消息塞进 context，再一股脑喂回去的例子

panels  = [] # 需要输出的部分
context = [] # 上下文信息

import panel as pn # GUI
pn.extension()
inp     = pn.widgets.Textinput(value='Hi', placeholder='Eneter text...')
btn_conversation = pn.widgets.Button(name='Chat!')
interactive_conversation = pn.bind(collect_msg, btn_conversation)
dashboard = pn.Column(
    inp,
    pn.Row(btn_conversation),
    pn.panel(interactive_conversation, loading_indicator=True)
)

dashboard # 显示 GUI

In [ ]:
def collect_msg(_):
    # 从输入读取 prompt，追加到 context
    prompt = inp.value_input
    inp.value = ''
    context.append({'role': 'user', 'content': prompt})
    # 把 context 喂给 LLM，拿到 response（也塞进 context）
    response = get_completion_from_message(context)
    context.append({'role': 'assistant', 'content': response})
    # 一些可视化输出
    panels.append(pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(pn.Row('Assistant:', pn.pane.Markdown(response, width=600)))
    
    return pn.Column(*panels)

In [ ]:
# 我们也可以尝试插入一些 system msg 来进行突击检查
msgs = context.copy() # 需要保留一些上下文
msgs.append({
    'role': 'system',
    'prompt': f"""
    Create a JSON summary of the previous food order. \
    Itemize the price for each item. 
    The fields should be 1) pizza, include size 2) list of toppings \
    3) list of drinks, include size 4) list of sides, include size 5) total price.
    """
})
# 因为我们希望输出稳定可靠，所以这里用 t = 0
response = get_completion_from_message(msgs, temperature=0)

SyntaxError: EOL while scanning string literal (1343602321.py, line 5)